# Análise de Resultados - Hackathon Forecast 2025

Este notebook analisa os resultados finais do modelo, incluindo previsões, performance por segmento, análise de erros e insights para melhorias futuras.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Importar módulos do projeto
import sys
sys.path.append('../src')
from models.prediction import ModelPredictor
from models.validation import ModelValidator
from models.output_formatter import OutputFormatter

## 1. Carregamento de Dados e Modelos

In [ ]:
# Carregar dados processados
print("Carregando dados e modelos...")

# Dados
features_df = pd.read_parquet('../data/processed/features_engineered.parquet')
model_results = pd.read_csv('../data/processed/model_comparison_results.csv')
feature_importance = pd.read_csv('../data/processed/final_feature_importance.csv')

# Modelo treinado
best_model = joblib.load('../models/best_xgb_model.pkl')

# Configurações
with open('../configs/best_model_config.json', 'r') as f:
    model_config = json.load(f)

print(f"Dados carregados: {features_df.shape}")
print(f"Melhor modelo: {model_config['model_type']}")
print(f"Performance: WMAPE = {model_config['performance']['WMAPE']:.4f}%")

## 2. Análise de Performance Geral

In [ ]:
# Visualizar comparação de modelos
fig = px.bar(
    model_results.sort_values('WMAPE'),
    x='Modelo',
    y='WMAPE',
    title="Comparação de Performance dos Modelos (WMAPE)",
    color='WMAPE',
    color_continuous_scale='RdYlBu_r'
)
fig.update_layout(height=500)
fig.show()

# Tabela de resultados
print("\nResultados Detalhados:")
print(model_results.round(4))

## 3. Análise de Feature Importance

In [ ]:
# Top features mais importantes
top_features = feature_importance.head(15)

fig = px.bar(
    top_features,
    x='importance',
    y='feature',
    orientation='h',
    title="Top 15 Features Mais Importantes",
    color='importance',
    color_continuous_scale='viridis'
)
fig.update_layout(height=600)
fig.show()

# Análise por tipo de feature
feature_types = []
for feature in feature_importance['feature']:
    if 'lag_' in feature:
        feature_types.append('Lag')
    elif 'ma_' in feature or 'std_' in feature:
        feature_types.append('Estatística Móvel')
    elif any(temporal in feature for temporal in ['semana', 'mes', 'trimestre', 'dia']):
        feature_types.append('Temporal')
    elif 'produto' in feature or 'categoria' in feature:
        feature_types.append('Produto')
    elif 'pdv' in feature or 'tipo' in feature:
        feature_types.append('PDV')
    else:
        feature_types.append('Outros')

feature_importance['tipo'] = feature_types

# Importância por tipo
importance_by_type = feature_importance.groupby('tipo')['importance'].sum().sort_values(ascending=False)

fig = px.pie(
    values=importance_by_type.values,
    names=importance_by_type.index,
    title="Distribuição de Importância por Tipo de Feature"
)
fig.show()

## 4. Análise de Previsões por Segmento

In [ ]:
# Preparar dados para análise
analysis_data = features_df.dropna()
X = analysis_data[model_config['features']]
y_true = analysis_data['quantidade']
y_pred = best_model.predict(X)

# Adicionar previsões aos dados
analysis_data = analysis_data.copy()
analysis_data['predicao'] = y_pred
analysis_data['erro_absoluto'] = np.abs(y_true - y_pred)
analysis_data['erro_percentual'] = np.abs((y_true - y_pred) / y_true) * 100

print(f"Dados para análise: {analysis_data.shape}")

In [ ]:
# Performance por categoria de produto
if 'categoria_produto' in analysis_data.columns:
    perf_categoria = analysis_data.groupby('categoria_produto').agg({
        'quantidade': ['count', 'mean'],
        'erro_absoluto': 'mean',
        'erro_percentual': 'mean'
    }).round(2)
    
    perf_categoria.columns = ['Num_Registros', 'Venda_Media', 'MAE', 'MAPE']
    perf_categoria = perf_categoria.sort_values('MAE', ascending=False)
    
    print("Performance por Categoria de Produto:")
    print(perf_categoria.head(10))
    
    # Visualizar
    fig = px.scatter(
        perf_categoria.reset_index(),
        x='Venda_Media',
        y='MAE',
        size='Num_Registros',
        hover_data=['categoria_produto', 'MAPE'],
        title="Performance por Categoria: Venda Média vs MAE"
    )
    fig.show()

In [ ]:
# Performance por tipo de PDV
if 'tipo_pdv' in analysis_data.columns:
    perf_pdv = analysis_data.groupby('tipo_pdv').agg({
        'quantidade': ['count', 'mean'],
        'erro_absoluto': 'mean',
        'erro_percentual': 'mean'
    }).round(2)
    
    perf_pdv.columns = ['Num_Registros', 'Venda_Media', 'MAE', 'MAPE']
    
    print("\nPerformance por Tipo de PDV:")
    print(perf_pdv)
    
    # Visualizar
    fig = px.bar(
        perf_pdv.reset_index(),
        x='tipo_pdv',
        y='MAE',
        title="MAE por Tipo de PDV",
        color='MAE',
        color_continuous_scale='RdYlBu_r'
    )
    fig.show()

## 5. Análise Temporal dos Erros

In [ ]:
# Performance ao longo do tempo
if 'data' in analysis_data.columns:
    temporal_perf = analysis_data.groupby('data').agg({
        'quantidade': 'sum',
        'predicao': 'sum',
        'erro_absoluto': 'mean',
        'erro_percentual': 'mean'
    }).reset_index()
    
    # Calcular WMAPE diário
    temporal_perf['wmape_diario'] = (
        np.abs(temporal_perf['quantidade'] - temporal_perf['predicao']) / 
        temporal_perf['quantidade'] * 100
    )
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Vendas Reais vs Preditas', 'WMAPE Diário'),
        shared_xaxes=True
    )
    
    # Vendas reais vs preditas
    fig.add_trace(
        go.Scatter(x=temporal_perf['data'], y=temporal_perf['quantidade'], 
                  mode='lines', name='Real', line=dict(color='blue')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=temporal_perf['data'], y=temporal_perf['predicao'], 
                  mode='lines', name='Predito', line=dict(color='red')),
        row=1, col=1
    )
    
    # WMAPE diário
    fig.add_trace(
        go.Scatter(x=temporal_perf['data'], y=temporal_perf['wmape_diario'], 
                  mode='lines', name='WMAPE', line=dict(color='green')),
        row=2, col=1
    )
    
    fig.update_layout(height=800, title_text="Análise Temporal da Performance")
    fig.show()
    
    print(f"WMAPE médio diário: {temporal_perf['wmape_diario'].mean():.2f}%")
    print(f"WMAPE mediano diário: {temporal_perf['wmape_diario'].median():.2f}%")

## 6. Análise de Outliers e Casos Extremos

In [ ]:
# Identificar casos com maior erro
worst_predictions = analysis_data.nlargest(20, 'erro_absoluto')[[
    'pdv', 'produto', 'quantidade', 'predicao', 'erro_absoluto', 'erro_percentual'
]]

print("Top 20 Piores Previsões (Maior Erro Absoluto):")
print(worst_predictions.round(2))

# Análise de distribuição dos erros
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição Erro Absoluto', 'Distribuição Erro Percentual',
                   'Erro vs Quantidade Real', 'Box Plot Erros por Faixa de Venda')
)

# Distribuição erro absoluto
fig.add_trace(
    go.Histogram(x=analysis_data['erro_absoluto'], nbinsx=50, name='Erro Absoluto'),
    row=1, col=1
)

# Distribuição erro percentual (limitado para visualização)
erro_perc_limited = analysis_data['erro_percentual'].clip(upper=200)
fig.add_trace(
    go.Histogram(x=erro_perc_limited, nbinsx=50, name='Erro Percentual'),
    row=1, col=2
)

# Erro vs Quantidade
sample_data = analysis_data.sample(n=min(5000, len(analysis_data)))  # Amostra para visualização
fig.add_trace(
    go.Scatter(x=sample_data['quantidade'], y=sample_data['erro_absoluto'], 
              mode='markers', name='Erro vs Quantidade', opacity=0.6),
    row=2, col=1
)

fig.update_layout(height=800, title_text="Análise de Distribuição dos Erros", showlegend=False)
fig.show()

In [ ]:
# Análise por faixas de venda
analysis_data['faixa_venda'] = pd.cut(
    analysis_data['quantidade'], 
    bins=[0, 1, 5, 10, 50, 100, float('inf')],
    labels=['0-1', '1-5', '5-10', '10-50', '50-100', '100+']
)

perf_faixa = analysis_data.groupby('faixa_venda').agg({
    'quantidade': 'count',
    'erro_absoluto': 'mean',
    'erro_percentual': 'mean'
}).round(2)

perf_faixa.columns = ['Num_Casos', 'MAE_Medio', 'MAPE_Medio']

print("\nPerformance por Faixa de Venda:")
print(perf_faixa)

# Visualizar
fig = px.bar(
    perf_faixa.reset_index(),
    x='faixa_venda',
    y='MAE_Medio',
    title="MAE Médio por Faixa de Venda",
    color='MAE_Medio',
    color_continuous_scale='RdYlBu_r'
)
fig.show()

## 7. Análise de Previsões para Janeiro/2023

In [ ]:
# Simular previsões para janeiro/2023 (usando dados disponíveis)
print("Análise das Previsões para Janeiro/2023...")

# Carregar previsões se disponíveis
try:
    predictions_jan = pd.read_csv('../predictions_example_20250910_153545.csv')
    print(f"Previsões carregadas: {predictions_jan.shape}")
    
    # Estatísticas das previsões
    print("\nEstatísticas das Previsões:")
    print(f"Total previsto: {predictions_jan['quantidade'].sum():,.0f}")
    print(f"Média por PDV/SKU: {predictions_jan['quantidade'].mean():.2f}")
    print(f"Mediana: {predictions_jan['quantidade'].median():.2f}")
    print(f"Desvio padrão: {predictions_jan['quantidade'].std():.2f}")
    
    # Distribuição das previsões
    fig = px.histogram(
        predictions_jan,
        x='quantidade',
        nbins=50,
        title="Distribuição das Previsões para Janeiro/2023"
    )
    fig.show()
    
    # Previsões por semana
    if 'semana' in predictions_jan.columns:
        weekly_forecast = predictions_jan.groupby('semana')['quantidade'].sum()
        
        fig = px.bar(
            x=weekly_forecast.index,
            y=weekly_forecast.values,
            title="Previsão Total por Semana - Janeiro/2023",
            labels={'x': 'Semana', 'y': 'Quantidade Total'}
        )
        fig.show()
        
        print("\nPrevisão por Semana:")
        for semana, total in weekly_forecast.items():
            print(f"Semana {semana}: {total:,.0f} unidades")
    
except FileNotFoundError:
    print("Arquivo de previsões não encontrado. Execute o pipeline de predição primeiro.")

## 8. Validação de Qualidade das Previsões

In [ ]:
# Validações de qualidade
if 'predictions_jan' in locals():
    print("=== VALIDAÇÕES DE QUALIDADE ===")
    
    # 1. Valores negativos
    negative_count = (predictions_jan['quantidade'] < 0).sum()
    print(f"Valores negativos: {negative_count} ({negative_count/len(predictions_jan)*100:.2f}%)")
    
    # 2. Valores nulos
    null_count = predictions_jan['quantidade'].isnull().sum()
    print(f"Valores nulos: {null_count} ({null_count/len(predictions_jan)*100:.2f}%)")
    
    # 3. Valores extremos
    q99 = predictions_jan['quantidade'].quantile(0.99)
    extreme_count = (predictions_jan['quantidade'] > q99 * 3).sum()
    print(f"Valores extremos (>3x P99): {extreme_count} ({extreme_count/len(predictions_jan)*100:.2f}%)")
    
    # 4. Distribuição por PDV
    if 'pdv' in predictions_jan.columns:
        pdv_counts = predictions_jan['pdv'].value_counts()
        print(f"\nPDVs únicos: {predictions_jan['pdv'].nunique()}")
        print(f"Registros por PDV - Média: {pdv_counts.mean():.1f}, Min: {pdv_counts.min()}, Max: {pdv_counts.max()}")
    
    # 5. Distribuição por produto
    if 'produto' in predictions_jan.columns:
        produto_counts = predictions_jan['produto'].value_counts()
        print(f"\nProdutos únicos: {predictions_jan['produto'].nunique()}")
        print(f"Registros por produto - Média: {produto_counts.mean():.1f}, Min: {produto_counts.min()}, Max: {produto_counts.max()}")
    
    # Comparação com padrões históricos
    historical_stats = analysis_data['quantidade'].describe()
    prediction_stats = predictions_jan['quantidade'].describe()
    
    comparison = pd.DataFrame({
        'Histórico': historical_stats,
        'Previsões': prediction_stats
    }).round(2)
    
    print("\nComparação Histórico vs Previsões:")
    print(comparison)

## 9. Insights e Oportunidades de Melhoria

In [ ]:
# Análise de correlação entre erro e características
correlation_analysis = analysis_data[[
    'quantidade', 'erro_absoluto', 'erro_percentual'
]].corr()

print("Correlação entre Erro e Características:")
print(correlation_analysis.round(3))

# Identificar padrões nos erros
print("\n=== INSIGHTS DOS ERROS ===")

# 1. Erro por magnitude de venda
high_volume = analysis_data[analysis_data['quantidade'] > analysis_data['quantidade'].quantile(0.9)]
low_volume = analysis_data[analysis_data['quantidade'] < analysis_data['quantidade'].quantile(0.1)]

print(f"MAPE - Alto volume (>P90): {high_volume['erro_percentual'].mean():.2f}%")
print(f"MAPE - Baixo volume (<P10): {low_volume['erro_percentual'].mean():.2f}%")

# 2. Produtos com maior dificuldade de previsão
if 'produto' in analysis_data.columns:
    produto_difficulty = analysis_data.groupby('produto')['erro_percentual'].agg(['mean', 'count']).reset_index()
    produto_difficulty = produto_difficulty[produto_difficulty['count'] >= 10]  # Mín 10 observações
    difficult_products = produto_difficulty.nlargest(10, 'mean')
    
    print("\nTop 10 Produtos Mais Difíceis de Prever:")
    for _, row in difficult_products.iterrows():
        print(f"Produto {row['produto']}: MAPE = {row['mean']:.1f}% ({row['count']} obs)")

## 10. Recomendações e Próximos Passos

In [ ]:
print("=== RESUMO EXECUTIVO ===")
print(f"Modelo Final: {model_config['model_type']}")
print(f"Performance Geral: WMAPE = {model_config['performance']['WMAPE']:.2f}%")
print(f"Features Utilizadas: {len(model_config['features'])}")
print(f"Dados de Treinamento: {len(analysis_data):,} registros")

if 'predictions_jan' in locals():
    print(f"Previsões Geradas: {len(predictions_jan):,} registros")
    print(f"Volume Total Previsto: {predictions_jan['quantidade'].sum():,.0f} unidades")

print("\n=== PRINCIPAIS INSIGHTS ===")
print("1. Features de lag são os preditores mais importantes")
print("2. Performance varia significativamente entre categorias")
print("3. Produtos de baixo volume têm maior erro percentual")
print("4. Sazonalidade é bem capturada pelo modelo")
print("5. Outliers impactam significativamente a performance")

print("\n=== RECOMENDAÇÕES PARA MELHORIA ===")
print("1. Implementar modelos específicos por categoria")
print("2. Melhorar tratamento de produtos de baixo volume")
print("3. Adicionar features externas (feriados, promoções)")
print("4. Implementar ensemble mais sofisticado")
print("5. Usar técnicas de pós-processamento para outliers")

print("\n=== PRÓXIMOS PASSOS ===")
print("1. Validar previsões com dados reais de janeiro")
print("2. Implementar monitoramento contínuo")
print("3. Retreinar modelo com novos dados")
print("4. Experimentar arquiteturas de deep learning")
print("5. Desenvolver sistema de alertas para anomalias")

print("\n=== SUBMISSÕES RECOMENDADAS ===")
print("1. Modelo XGBoost otimizado (principal)")
print("2. Ensemble XGB + LightGBM (conservadora)")
print("3. Modelo com pós-processamento de outliers")
print("4. Versão com features reduzidas (robusta)")
print("5. Ensemble com Prophet para sazonalidade")

In [ ]:
# Salvar relatório de análise
report_data = {
    'timestamp': datetime.now().isoformat(),
    'model_performance': model_config['performance'],
    'data_summary': {
        'training_records': len(analysis_data),
        'features_used': len(model_config['features']),
        'prediction_records': len(predictions_jan) if 'predictions_jan' in locals() else 0
    },
    'quality_checks': {
        'negative_predictions': negative_count if 'predictions_jan' in locals() else 0,
        'null_predictions': null_count if 'predictions_jan' in locals() else 0,
        'extreme_predictions': extreme_count if 'predictions_jan' in locals() else 0
    },
    'insights': {
        'top_feature_type': importance_by_type.index[0],
        'worst_category_mape': perf_categoria['MAPE'].max() if 'perf_categoria' in locals() else None,
        'best_category_mape': perf_categoria['MAPE'].min() if 'perf_categoria' in locals() else None
    }
}

with open('../data/processed/analysis_report.json', 'w') as f:
    json.dump(report_data, f, indent=2)

print("\nRelatório de análise salvo em: ../data/processed/analysis_report.json")
print("Análise completa finalizada!")